In [1]:
import numpy as np
import jupyter_black


jupyter_black.load()

# The Discrete Fourier Transform
The FFT is a fast $\mathcal{O}[NlogN]$ algorithm to compute the Discrete Fourier Transform DFT, which naively is an $\mathcal{0}[N^2]$ computation.

#### Forward Discrete Fourier Transform (DFT):

\begin{align}
X_k = \sum_{n=0}^{N-1}x_n . e^{\frac{-i2\pi kn}{N}}
\end{align}

##### Inverse Discrete Fourier Transform (IDFT):

\begin{align}
X_n = \frac{1}{N}\sum_{k=0}^{N-1}X_k e^{\frac{i 2\pi k n}{N}}
\end{align}


This forward component of this boils down to a linear operation on $x$:

\begin{align}
\overrightarrow{X} = M . \overrightarrow{x}
\end{align}

with $M$ given by:
\begin{align}
M_{kn} = e^{\frac{-i 2\pi k n}{N}}
\end{align}


In [2]:
def DFT_python(x):
    x = np.asarray(x, dtype=float)
    N = x.shape[0]
    n = np.arange(N)
    k = n.reshape((N, 1))
    M = np.exp(-2j * np.pi * k * n / N)
    return np.dot(M, x)

In [3]:
x = np.random.random(1024)
np.allclose(DFT_python(x), np.fft.fft(x))

True

In [4]:
%timeit DFT_python(x)
%timeit np.fft.fft(x)

51.1 ms ± 1.81 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
11.3 µs ± 504 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


There are a miriad of different reasons why our python version is ~5000 times slower than the numpy method, from its implementation as a series of python operations on numpy objects (though the numerical heavy lifting is done by numpy, which is very rapid).
The primary reason is that the numpy version is fundamentally an $\mathcal{O}[NlogN]$ solution, where as the DFT implemented above is an $\mathcal{O}[N^2]$.

## Symmetries in the DFT

\begin{align}
X_{N+k} &= \sum_{n=0}^{N-1}x_n . e^{i 2\pi (N+k) n/N} \\
&= \sum_{n=0}^{N-1}x_n . e^{i 2\pi n}.e^{-i 2\pi k n / N} \\
&= \sum_{n=0}^{N-1}x_n . e^{-i2\pi k n / N}
\end{align}


## DFT to FFT

\begin{align}
X_k &= \sum_{n=0}^{N-1}x_n . e^{-i2\pi k n /N} \\
    &= \sum_{m=0}^{N/2 - 1} x_{2m} . e^{-i2\pi k (2m) / N} + \sum_{m=0}^{N/2-1}x_{2m+1} . e^{-i2\pi k (2m + 1) / N} \\
    &= \sum_{m=0}^{N/2 - 1} x_{2m} . e^{-i2\pi k m / (N/2)} + e^{-i2\pi k/N}\sum_{m=0}^{N/2 - 1} x_{2m+1} . e^{-i2\pi k m / (N/2)}\\
\end{align}

In [17]:
def FFT(x):
    x = np.asarray(x, dtype=float)
    N = x.shape[0]
    if N % 2 > 0:
        raise ValueError("Size of x must be a power of 2")
    elif N <= 32:
        return DFT_python(x)
    else:
        X_even = FFT(x[::2])
        X_odd = FFT(x[1::2])
        factor = np.exp(-2j * np.pi * np.arange(N) / N)
        return np.concatenate(
            [X_even + factor[: N // 2] * X_odd, X_even + factor[N // 2 :] * X_odd]
        )

In [18]:
x = np.random.random(1024)
np.allclose(FFT(x), np.fft.fft(x))

True

In [19]:
%timeit DFT_python(x)
%timeit FFT(x)
%timeit np.fft.fft(x)

59.5 ms ± 2.61 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
2.31 ms ± 320 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
11.6 µs ± 288 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [20]:
def FFT_vectorised(x):
    x = np.asarray(x, dtype=float)
    N = x.shape[0]

    if np.log2(N) % 1 > 0:
        raise ValueError("size of x must be a power of 2")

    N_min = min(N, 32)

    n = np.arange(N_min)
    k = n[:, None]
    M = np.exp(-2j * np.pi * n * k / N_min)
    X = np.dot(M, x.reshape((N_min, -1)))

    while X.shape[0] < N:
        X_even = X[:, : X.shape[1] // 2]
        X_odd = X[:, X.shape[1] // 2 :]
        factor = np.exp(-1j * np.pi * np.arange(X.shape[0]) / X.shape[0])[:, None]
        X = np.vstack([X_even + factor * X_odd, X_even - factor * X_odd])

    return X.ravel()

In [21]:
x = np.random.random(1024)
np.allclose(FFT_vectorised(x), np.fft.fft(x))

True

In [22]:
%timeit DFT_python(x)
%timeit FFT(x)
%timeit FFT_vectorised(x)
%timeit np.fft.fft(x)

57.1 ms ± 2.97 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
2.1 ms ± 85.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
226 µs ± 22.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
11.8 µs ± 370 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
